# QuantFreedom Video - Feb 14, 2024 - Building a Trading Bot

In [5]:
# YT Video - https://youtu.be/ZqtJgyCR0Qo?si=UUSQ146qR5ML_ca7
# import libs
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import NamedTuple
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import macd_tv, ema_tv

# from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
# from numba import njit
np.set_printoptions(formatter={'float_kind': '{:0.2f}'.format})
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('MFX_KEY')
api_secret = os.getenv('MFX_SECRET')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Download candles frrom exchange
candles = dl_ex_candles(
  exchange='mufex',
  symbol='BTCUSDT',
  timeframe='5m',
  candles_to_dl=1500,
)
print(candles)

datetimes = candles[:, CandleBodyType.Timestamp].astype('datetime64[ms]')
closing_prices = candles[:, CandleBodyType.Close]
low_prices = candles[:, CandleBodyType.Low]

[[1710321600000.00 73523.30 73523.30 73463.10 73496.30 43.43]
 [1710321900000.00 73496.30 73585.00 73487.70 73539.80 27.66]
 [1710322200000.00 73539.80 73539.80 73356.50 73407.10 58.75]
 ...
 [1710770700000.00 67462.90 67492.90 67316.30 67374.90 89.28]
 [1710771000000.00 67374.90 67801.20 67374.90 67788.70 150.64]
 [1710771300000.00 67788.70 67875.50 67451.60 67474.40 115.33]]


In [11]:
# Define Indicators Length
macd_below = 0
fast_length = 12
slow_length = 26
signal_smoothing = 9
ema_length = 200


In [12]:
# MACD 
histogram, macd, signal = macd_tv(
  source=closing_prices,
  fast_length=fast_length,
  slow_length=slow_length,
  signal_smoothing=signal_smoothing

)
macd

array([nan, nan, nan, ..., -215.80, -201.44, -212.97])

In [13]:
# EMA
ema = ema_tv(
  source=closing_prices,
  length=ema_length
)
ema

array([nan, nan, nan, ..., 68078.92, 68076.04, 68070.05])

In [14]:
# Define MACD 
prev_macd = np.roll(macd, 1)
prev_macd[0] = np.nan

prev_signal = np.roll(signal, 1)
prev_signal[0] = np.nan

In [20]:
# When MACD crosses above our signal == BUY
macd_below_signal = prev_macd < prev_signal
macd_above_signal = macd > signal
low_price_below_ema = low_prices > ema
macd_below_number = macd < macd_below

macd_below_signal # We get array of True/False

array([False, False, False, ...,  True,  True,  True])

In [21]:
entries = (
  (low_price_below_ema == True)
  & (macd_below_signal == True)
  & (macd_above_signal == True)
  & (macd_below_number == True)
)
entries

array([False, False, False, ..., False, False, False])

In [22]:
entry_signals =  np.where(entries, macd, np.nan)

In [23]:
# Plot our entries, to see how profitable strat is
fig = go.Figure()
fig = make_subplots(
  cols=1,
  rows=2,
  shared_xaxes=True,
  subplot_titles=['Candles', 'MACD'],
  row_heights=[0.6, 0.4],
  vertical_spacing=0.1,
)
# Chart CandleSticks
fig.append_trace(
  go.Candlestick(
    x=datetimes,
    open=candles[:, CandleBodyType.Open],
    high=candles[:, CandleBodyType.High],
    low=candles[:, CandleBodyType.Low],
    close=candles[:, CandleBodyType.Close],
    name='Candles',
  ),
  col=1,
  row=1,
)
fig.append_trace(
  go.Scatter(
    x=datetimes,
    y=ema,
    name='EMA',
    line_color='yellow',
  ),
  col=1,
  row=1,
)
ind_shift = np.roll(histogram, 1)
ind_shift[0] = np.nan
colors = np.where(
  histogram >= 0,
  np.where(ind_shift < histogram, '#26A69A', '#B2DFD8'),
  np.where(ind_shift < histogram, '#FFCDD2', '#FF5252'),
)
fig.append_trace(
  go.Bar(
    x=datetimes,
    y=histogram,
    name='histogram',
    marker_color=colors,
  ),
  
)
